In [1]:
import numpy as np
import pandas as pd

### Preparing the data

Appliances:<br>
Washing machine: 5<br>
Dishwasher: 6<br>
Kettle: 10<br>
Fridge: 12<br>
Microwave: 13<br>

In [4]:
house1_aggregate = pd.read_csv('../master_data/house_1/channel_1.dat', header = None,
                                    names = ["Time","Power"], delim_whitespace = True,
                                    index_col = 0)
#wm_power = pd.read_csv('house_1/channel_5.dat', header = None,
#                                    names = ["Time","Power"], delim_whitespace = True,
#                                    index_col = 0)
#wm_button = pd.read_csv('house_1/channel_5_button_press.dat', header = None,
#                                    names = ["Time","Power"], delim_whitespace = True,
#                                    index_col = 0)
#dw_power = pd.read_csv('house_1/channel_6.dat', header = None,
#                                    names = ["Time","Power"], delim_whitespace = True,
#                                    index_col = 0)
#dw_button = pd.read_csv('house_1/channel_6_button_press.dat', header = None,
#                                    names = ["Time","Power"], delim_whitespace = True,
#                                    index_col = 0)
#kettle_power = pd.read_csv('../master_data/house_1/channel_10.dat', header = None,
#                                    names = ["Time","Power"], delim_whitespace = True,
#                                    index_col = 0)
#kettle_button = pd.read_csv('../master_data/house_1/channel_10_button_press.dat', header = None,
#                                    names = ["Time","Power"], delim_whitespace = True,
#                                    index_col = 0)
fridge_power = pd.read_csv('../master_data/house_1/channel_12.dat', header = None,
                                    names = ["Time","Power"], delim_whitespace = True,
                                    index_col = 0)
fridge_button = pd.read_csv('../master_data/house_1/channel_12_button_press.dat', header = None,
                                    names = ["Time","Power"], delim_whitespace = True,
                                    index_col = 0)
#mw_power = pd.read_csv('../master_data/house_1/channel_13.dat', header = None,
#                                    names = ["Time","Power"], delim_whitespace = True,
#                                    index_col = 0)
#mw_button = pd.read_csv('../master_data/house_1/channel_13_button_press.dat', header = None,
#                                    names = ["Time","Power"], delim_whitespace = True,
#                                    index_col = 0)


/home/bjornif/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


The button data seems to have been recorded three months later than the signal data.

In [5]:
#Extract activations from the appliance signal data. The default optional parameters are the same used for
#microwave activations by Kelly&Knottenbelt. Takes a while.

def get_activations(power_series, min_off_duration=30, min_on_duration=12,
                        border=1, power_threshold=200):
    activations = []
    signal_start = None
    signal_end = None
    no_signal = 0
    current_signal = []
    series_length = len(power_series)
    series_length_percent = int(len(power_series)/100)
    measurements_processed = 0
    
    power_series.index = pd.to_datetime(power_series.index,unit='s')
    min_off_duration = pd.Timedelta(str(min_off_duration) + 's')
    min_on_duration = pd.Timedelta(str(min_off_duration) + 's')
    timestep = pd.Timedelta('6s')
    
    for time in power_series.index:
        measurement = power_series.loc[time]['Power']
        measurements_processed +=1
        
        if measurements_processed % series_length_percent == 0:
            print(str(measurements_processed/series_length_percent) + '%')
        
        if (measurement >= power_threshold):
            if signal_start is None:
                signal_start = time - timestep
            no_signal = 0
            current_signal.append(measurement)
            signal_end = time + timestep
            
        else:
            if signal_end is not None:
                no_signal = time - signal_end
                if (no_signal >= min_off_duration) and (signal_start is not None):
                    if (signal_end - signal_start) > min_on_duration:
                        mean_power = np.mean(current_signal)
                        activations.append([signal_start,signal_end,mean_power])
                    current_signal = []
                    signal_start = None
                    signal_end = None
    
    return activations

In [7]:
fridge_activations = get_activations(fridge_power, power_threshold=50, min_on_duration=60, min_off_duration=12)


1.0%
2.0%
3.0%
4.0%
5.0%
6.0%
7.0%
8.0%
9.0%
10.0%
11.0%
12.0%
13.0%
14.0%
15.0%
16.0%
17.0%
18.0%
19.0%
20.0%
21.0%
22.0%
23.0%
24.0%
25.0%
26.0%
27.0%
28.0%
29.0%
30.0%
31.0%
32.0%
33.0%
34.0%
35.0%
36.0%
37.0%
38.0%
39.0%
40.0%
41.0%
42.0%
43.0%
44.0%
45.0%
46.0%
47.0%
48.0%
49.0%
50.0%
51.0%
52.0%
53.0%
54.0%
55.0%
56.0%
57.0%
58.0%
59.0%
60.0%
61.0%
62.0%
63.0%
64.0%
65.0%
66.0%
67.0%
68.0%
69.0%
70.0%
71.0%
72.0%
73.0%
74.0%
75.0%
76.0%
77.0%
78.0%
79.0%
80.0%
81.0%
82.0%
83.0%
84.0%
85.0%
86.0%
87.0%
88.0%
89.0%
90.0%
91.0%
92.0%
93.0%
94.0%
95.0%
96.0%
97.0%
98.0%
99.0%
100.0%


In [8]:
fridge_activations_pd = pd.DataFrame(fridge_activations)
fridge_activations_pd.to_csv('../master_data/nilm/fridge_activations.dat', sep = ' ')

In [ ]:
#fridge_activations = get_activations(fridge_power)
#wm_activations = get_activations(wm_power)
#dw_activations = get_activations(dw_power)
#mw_activations = get_activations(mw_power)


In [ ]:
#fridge_activations_pd = pd.DataFrame(fridge_activations)
#wm_activations_pd = pd.DataFrame(wm_activations)
#kettle_activations_pd = pd.DataFrame(kettle_activations)
#dw_activations_pd = pd.DataFrame(dw_activations)
#dw_activations_pd.to_csv('dw_activations.dat', sep = ' ')
#wm_activations_pd.to_csv('wm_activations.dat', sep = ' ')
#fridge_activations_pd.to_csv('fridge_activations.dat', sep = ' ')
#kettle_activations_pd.to_csv('kettle_activations.dat', sep = ' ')

Done.